In [2]:
import time
import pandas as pd
import numpy as np
from pyecharts.charts import *
import pyecharts.options as opts
from pyecharts.commons.utils import JsCode

import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('./医疗数据.csv', encoding="gbk")
data.head(3)

,ID,NL,XB,RYLB,YLLB,RYQH,JZQH,JGDJ,RYRQ,CYRQ,ZDMC,ZFY,TCFY,JSRQ
0,1001833932,91.0,男,离休,普通住院,费县,山东省临沂市,三级医院,1506787200,1507737600,左心衰竭,6641.21,6574.07,1516204800
1,1002064861,42.0,女,居民,转省外住院,兰山区,山东省临沂市,三级医院,1515945600,1516636800,左乳癌新辅助化疗后,8129.72,1883.29,1516636800
2,1015268667,12.0,男,居民,转省外住院,双堠镇,山东省临沂市,三级医院,1506787200,1509724800,阻塞性睡眠呼吸暂停综合征,4539.06,1751.01,1516809600


In [5]:
# 删除有缺失项的行 并在原数据上修改
data.dropna(axis=0, inplace=True)

In [8]:
if data.duplicated().sum() == 0:
    print("数据未存在重复")
else:
    num = data.duplicated().sum()
    data.drop_duplicates(inplace=True, keep='first')
    print('已处理{}个重复值数据'.format(num))

已处理14个重复值数据


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16455 entries, 0 to 16564
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      16455 non-null  int64  
 1   NL      16455 non-null  float64
 2   XB      16455 non-null  object 
 3   RYLB    16455 non-null  object 
 4   YLLB    16455 non-null  object 
 5   RYQH    16455 non-null  object 
 6   JZQH    16455 non-null  object 
 7   JGDJ    16455 non-null  object 
 8   RYRQ    16455 non-null  int64  
 9   CYRQ    16455 non-null  int64  
 10  ZDMC    16455 non-null  object 
 11  ZFY     16455 non-null  float64
 12  TCFY    16455 non-null  float64
 13  JSRQ    16455 non-null  int64  
dtypes: float64(3), int64(4), object(7)
memory usage: 1.9+ MB


In [19]:
# 时间数据处理
def trans_time(time_stamp):
    timeStamp = time_stamp
    # 将时间戳转换为本地时间的数组表示
    timeArray = time.localtime(timeStamp)
    Time = time.strftime('%Y-%m-%d %H:%M:%S', timeArray)
    print(Time, type(Time))
    return Time

In [23]:
data['RYRQ'] = data['RYRQ'].map(lambda x:trans_time(x))
data['CYRQ'] = data['CYRQ'].map(lambda x:trans_time(x))
data['JSRQ'] = data['JSRQ'].map(lambda x:trans_time(x))
data.head(3)

,ID,NL,XB,RYLB,YLLB,RYQH,JZQH,JGDJ,RYRQ,CYRQ,ZDMC,ZFY,TCFY,JSRQ
0,1001833932,91.0,男,离休,普通住院,费县,山东省临沂市,三级医院,2017-10-01 00:00:00,2017-10-12 00:00:00,左心衰竭,6641.21,6574.07,2018-01-18 00:00:00
1,1002064861,42.0,女,居民,转省外住院,兰山区,山东省临沂市,三级医院,2018-01-15 00:00:00,2018-01-23 00:00:00,左乳癌新辅助化疗后,8129.72,1883.29,2018-01-23 00:00:00
2,1015268667,12.0,男,居民,转省外住院,双堠镇,山东省临沂市,三级医院,2017-10-01 00:00:00,2017-11-04 00:00:00,阻塞性睡眠呼吸暂停综合征,4539.06,1751.01,2018-01-25 00:00:00


In [26]:
data["NL_CUT"] = pd.cut(data['NL'], bins=[0,6,12,17,45,69,1000], labels=['婴幼儿', '少儿', '青少年', '青年', '中年', '老年'], right=True)
data.drop('ID', axis=1, inplace=True)
data.head(3)

,NL,XB,RYLB,YLLB,RYQH,JZQH,JGDJ,RYRQ,CYRQ,ZDMC,ZFY,TCFY,JSRQ,NL_CUT
0,91.0,男,离休,普通住院,费县,山东省临沂市,三级医院,2017-10-01 00:00:00,2017-10-12 00:00:00,左心衰竭,6641.21,6574.07,2018-01-18 00:00:00,老年
1,42.0,女,居民,转省外住院,兰山区,山东省临沂市,三级医院,2018-01-15 00:00:00,2018-01-23 00:00:00,左乳癌新辅助化疗后,8129.72,1883.29,2018-01-23 00:00:00,青年
2,12.0,男,居民,转省外住院,双堠镇,山东省临沂市,三级医院,2017-10-01 00:00:00,2017-11-04 00:00:00,阻塞性睡眠呼吸暂停综合征,4539.06,1751.01,2018-01-25 00:00:00,少儿


In [35]:
tmp = data[data['XB'] == "男"]['ZDMC'].value_counts().head(10)
attrs = tmp.index.tolist()
value = tmp.values.tolist()
attrs, value

(['冠心病',
  '糖尿病',
  '高血压',
  '尿毒症患者的透析治疗',
  '恶性肿瘤门诊放化疗',
  '脑梗塞后遗症期',
  '急性上呼吸道感染',
  '脑梗塞',
  '乙型病毒性肝炎',
  '腰椎间盘突出'],
 [1035, 1016, 840, 268, 206, 194, 186, 137, 127, 105])